In [1]:
import numpy as np
import rasterio as rio
from tslearn import clustering
from pathlib import Path
import matplotlib.pyplot as plt

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
ndviFpath = r"D:\Images\ndvi_corrected\ndvi_corrected_stack.tif"
with rio.open(ndviFpath) as dst:
    ndviImg = dst.read()
    ndviMeta = dst.meta
    ndviShape = ndviImg.shape


In [3]:
urbwaterFpath = r"D:\Images\ndvi_corrected\urban_water_mask.tif"
nonRabiFpath = r"D:\Images\clusterRabi\nonRabi_c50.tif"

In [4]:
with rio.open(urbwaterFpath) as dst:
    urbwaterMask = dst.read(1)
    urbWaterMeta = dst.meta
with rio.open(nonRabiFpath) as dst:
    nRabiImg = dst.read(1)
    nRabiMeta = dst.meta


In [5]:
ndviImg[:,urbwaterMask==1] = 0
ndviImg[:,nRabiImg==1] = 0

In [6]:
n_clusters=50

In [7]:
# Remove 20230314 data
img = ndviImg[23]

In [8]:
plt.imshow(img)
plt.show()

In [9]:
flatImg = ndviImg.reshape(ndviShape[0],-1).T
km_dst = clustering.TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", n_init=3, n_jobs=5, max_iter=5,max_iter_barycenter=3,).fit(flatImg)

In [21]:
plt.clf()
plt.plot(flatImg[76776])
plt.show()

In [12]:
flatCluster = km_dst.labels_
out = flatCluster.reshape([1:])

In [16]:
# outFname = input(r"Output file name: ")
outFpath = r"D:\Images\ndvi_corrected\cluster_50\clusterOutput_1.tif"
# outFpath = Path(outfolder, outFname)
with rio.open(outFpath, 'w', **urbWaterMeta) as dst:
    dst.write(out,1)

In [14]:
plt.ioff()
outGraphFolder = r"D:\Images\ndvi_corrected\cluster_50"
i = 0
while i <= n_clusters:
    clusterIdxs = np.where(flatCluster==i)[0]
    randomIdxs = np.random.choice(clusterIdxs, size=100)
    for j in randomIdxs:
        plt.plot(flatImg[j])
    plotFpath = Path(outGraphFolder, f"cluster_{i}.jpg")
    plt.savefig(plotFpath)
    plt.clf()
    i+= 1


ValueError: 'a' cannot be empty unless no samples are taken